## Downloading and Transcribing a Youtube Clip

In [24]:
from faster_whisper import WhisperModel
import yt_dlp
import os

In [25]:
def download_audio(youtube_url, out_dir="downloads"):
    os.makedirs(out_dir, exist_ok=True)
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{out_dir}/%(id)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        }]
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=True)
        return f"{out_dir}/{info['id']}.mp3", info['id']
    
def transcribe(audio_path, model_size="medium", output_dir="transcripts"):
    model = WhisperModel(model_size, compute_type="int8")
    segments, _ = model.transcribe(audio_path)
    os.makedirs(output_dir, exist_ok=True)
    text_chunks = []
    for seg in segments:
        text_chunks.append({
            "start": seg.start,
            "end": seg.end,
            "text": seg.text
        })

    output_path = os.path.join(output_dir, os.path.basename(audio_path).replace(".mp3", ".txt"))
    with open(output_path, "w") as f:
        for chunk in text_chunks:
            f.write(f"{chunk['text']}\n")

    print(f"Transcription saved to: {output_path}")
    return None

In [26]:
download_audio('https://www.youtube.com/watch?v=EKkFOMzwMgc', out_dir = '/Users/jpoberhauser/Desktop/baseballCompanion/data/')

[youtube] Extracting URL: https://www.youtube.com/watch?v=EKkFOMzwMgc
[youtube] EKkFOMzwMgc: Downloading webpage
[youtube] EKkFOMzwMgc: Downloading tv client config
[youtube] EKkFOMzwMgc: Downloading tv player API JSON
[youtube] EKkFOMzwMgc: Downloading ios player API JSON
[youtube] EKkFOMzwMgc: Downloading m3u8 information
[info] EKkFOMzwMgc: Downloading 1 format(s): 251
[download] Destination: /Users/jpoberhauser/Desktop/baseballCompanion/data//EKkFOMzwMgc.webm
[download] 100% of   50.77MiB in 00:00:01 at 33.38MiB/s    
[ExtractAudio] Destination: /Users/jpoberhauser/Desktop/baseballCompanion/data//EKkFOMzwMgc.mp3
Deleting original file /Users/jpoberhauser/Desktop/baseballCompanion/data//EKkFOMzwMgc.webm (pass -k to keep)


('/Users/jpoberhauser/Desktop/baseballCompanion/data//EKkFOMzwMgc.mp3',
 'EKkFOMzwMgc')

This is example is a 24 minute youtube clip and it gets compeltey transcribed in ~13 minutes using the medium model and in using the small model ~5 minutes

In [27]:
transcribe('/Users/jpoberhauser/Desktop/baseballCompanion/data/EKkFOMzwMgc.mp3',
            model_size="small", 
            output_dir="/Users/jpoberhauser/Desktop/baseballCompanion/data/transcripts/")

Transcription saved to: /Users/jpoberhauser/Desktop/baseballCompanion/data/transcripts/EKkFOMzwMgc.txt


### Let's generate and store embeddings 

In [46]:
# !pip install langchain 

In [41]:
# !pip install sentence-transformers faiss-cpu
import faiss
import pickle

In [ ]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)

# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)

(3, 384)


In [ ]:
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)

array([[ 0.01919569,  0.12008531,  0.15959838, ..., -0.00536285,
        -0.08109499,  0.05021335],
       [-0.01869035,  0.04151865,  0.0743155 , ...,  0.00486595,
        -0.06190439,  0.0318751 ],
       [ 0.13650198,  0.08227322, -0.02526161, ...,  0.08762044,
         0.03045845, -0.01075751]], shape=(3, 384), dtype=float32)

tensor([[1.0000, 0.6660, 0.1046],
        [0.6660, 1.0000, 0.1411],
        [0.1046, 0.1411, 1.0000]])


## compare model.similarity to FAISS

* obviously its oerkill to use FAISS for three sentence embeddings, but we will need it for hundreds of thousands of text chunks in a real vectorDB. 

* just to make sure we get similar results, we run the code below, and indeed, the first sentence is most similar to the second one, and they both give a similairyt of _around_ 0.66

In [40]:
#### compare to FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 = Euclidean, or use IndexFlatIP for cosine
index.add(embeddings)

# Query similarity from emb1
D, I = index.search(embeddings[0].reshape(1, -1), k=3)

print("Indices of nearest neighbors:", I)
print("Distances:", D)

Indices of nearest neighbors: [[0 1 2]]
Distances: [[0.         0.66808915 1.7908318 ]]


In [ ]:
import glob
from langchain.schema import Document
all_transcripts = glob.glob('/Users/jpoberhauser/Desktop/baseballCompanion/data/transcripts/*.txt')
all_transcripts

['/Users/jpoberhauser/Desktop/baseballCompanion/data/transcripts/CFtDihdNwJk.txt',
 '/Users/jpoberhauser/Desktop/baseballCompanion/data/transcripts/b5-xIieufYs.txt',
 '/Users/jpoberhauser/Desktop/baseballCompanion/data/transcripts/EKkFOMzwMgc.txt']

In [105]:
all_raw_docs = []
for i in all_transcripts:
    with open(i, "r", encoding="utf-8") as f:
        text = f.read()  # `text` is one big string
    all_raw_docs.append([text.replace('\n', '')])

In [109]:
len(all_raw_docs)

3

In [110]:
documents = [Document(inner[0]) for inner in all_raw_docs]

## Langchain

In [117]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import  util
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents)

In [118]:
len(all_splits)

180

In [119]:
all_chunks = [x.page_content for x in all_splits]

In [120]:
embeddings = model.encode(all_chunks)

In [124]:
query = "The New York Mets are very good"
query_embedding = model.encode(query)

# 4. Compute cosine similarities
cos_scores = util.cos_sim(query_embedding, embeddings)[0]  # shape: (3,)

# 5. Find most similar
most_similar_idx = cos_scores.argmax()
print(f"Query: {query}")
print(f"Most similar: {all_chunks[most_similar_idx]}")
print(f"Scores: {cos_scores}")

Query: The New York Mets are very good
Most similar: York Mets, they went and got Juan Soto. They went and signed Francis Golden Door back. They've been bringing players in. Having the richest owner in the sport obviously helps that. If you go down the line, the Cubs made a big play for Tucker and the rest of the Astros team. They bring him over. Now, they're performing very well. The Dodgers have went out and signed everybody. The Padres and all the trades they've made for Michael King, Dylan Cease, you know, they've done a lot. Champions League of Giants have landed a bunch of free agents. That's been the difference. They've just got the good players when the good players were available. If you look at the American League team, name me a team that's done that. Yeah. There isn't a team that's went out and got the big player. The Yankees, they did that and they made it to the World Series. They went out and got Juan Soto. Then this year, you know, they went out and made some trades for